In [1]:
import numpy as np
import pandas as pd
import os
import statsmodels as sm
from sklearn.mixture import GaussianMixture
from matplotlib import pyplot as plt
from pmdarima.arima import auto_arima
import cvxpy as cp
import time
import datetime as dt
import matplotlib.dates as mdates
import yfinance as yf
import mosek

In [2]:
cp.installed_solvers()

['ECOS', 'ECOS_BB', 'MOSEK', 'OSQP', 'SCIPY', 'SCS']

In [5]:
#get data
all_files = os.listdir(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\sp500_data_features')
dates = pd.read_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\sp500_data_features\{}'.format(all_files[0])).Date
data_dict = {}
for file in all_files:
    df = pd.read_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\sp500_data_features\{}'.format(file))
    data_dict[file] = df


In [6]:
print(dates)

0       1995-01-05
1       1995-01-06
2       1995-01-09
3       1995-01-10
4       1995-01-11
           ...    
6844    2022-03-11
6845    2022-03-14
6846    2022-03-15
6847    2022-03-16
6848    2022-03-17
Name: Date, Length: 6849, dtype: object


In [13]:
#get S&P500 data
yf.Ticker('SPY').history(start="1995-01-04", end="2022-03-18").to_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\SP500.csv')

In [15]:
sp500_df = pd.read_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\SP500.csv')
sp500_close = sp500_df.Close.values[:]
print(sp500_close.shape)
print(sp500_df.Date)

(6850,)
0        1/4/1995
1        1/5/1995
2        1/6/1995
3        1/9/1995
4       1/10/1995
          ...    
6845    3/11/2022
6846    3/14/2022
6847    3/15/2022
6848    3/16/2022
6849    3/17/2022
Name: Date, Length: 6850, dtype: object


In [16]:
#get features
mu_vec = np.empty([len(dates), len(all_files)])
vw_mu = np.empty([len(dates), len(all_files)])
vw_rsi = np.empty([len(dates), len(all_files)])

for i in range(len(dates)):
    mu_daily = np.array([])
    vw_mu_daily = np.array([])
    vw_rsi_daily = np.array([])
    
    total_volume = 0
    for file in all_files:
        ret = data_dict[file].Return[i]
        volume = data_dict[file].Volume[i]
        rsi = data_dict[file].RSI[i]

        total_volume += volume
        mu_daily = np.append(mu_daily, ret)
        vw_mu_daily = np.append(vw_mu_daily, ret*volume)
        vw_rsi_daily = np.append(vw_rsi_daily, rsi*volume)
    
    mu_vec[i] = mu_daily
    vw_mu[i] = vw_mu_daily / total_volume
    vw_rsi[i] = vw_rsi_daily / total_volume

In [17]:
mu_vec.shape

(6849, 200)

In [3]:
gamma_vals = np.logspace(-2, 3, num=10)
print(np.round(gamma_vals, decimals=2))

[1.0000e-02 4.0000e-02 1.3000e-01 4.6000e-01 1.6700e+00 5.9900e+00
 2.1540e+01 7.7430e+01 2.7826e+02 1.0000e+03]


C:\Users\xyyh\AppData\Local\Temp/ipykernel_7636/1416494193.py:1: RuntimeWarning: invalid value encountered in log10
  np.log10(-2)


nan

In [18]:
def opt_w(mu, Sigma, n, slvr):
    w = cp.Variable(n)
    gamma = cp.Parameter(nonneg=True)
    ret = mu.T@w 
    risk = cp.quad_form(w, Sigma)
    prob = cp.Problem(cp.Maximize(ret - gamma*risk), 
               [cp.sum(w) == 1, 
                w >= 0])
    
    SAMPLES = 10
    gamma_vals = np.logspace(-2, 3, num=SAMPLES)
    
    w_val = None
    prev_sharpe = -1000
    for i in range(SAMPLES):
        gamma.value = gamma_vals[i]
        prob.solve(solver=slvr)
        risk_data = cp.sqrt(risk).value
        ret_data = ret.value
        sharpe = ret_data/risk_data
        if sharpe > prev_sharpe:
            w_val = w.value
        prev_sharpe = sharpe 
    return w_val


def backtest(n_state, feature, train_len, freq):
    '''
    n_state: the number of market states (int);
    feature: the feature used to cluster data points;
    train_len: the time span of training set (in years);
    freq: days between each training set (int).
    
    returns the portfolio net asset value vector
    '''
    train_days = int(train_len * 253)
    n_iter = int((len(feature) - train_days)/freq)+1
    
    nav = [1]
    opt_failed_dates = []
    
    #create an initial equally weighted portfolio in case the optimization problem failed
    w = np.ones(mu_vec.shape[1])/mu_vec.shape[1] 
    
    for i in range(n_iter):
        #get the data and the historical states:
        feature_temp = feature[i*freq:i*freq+train_days]
        mu_temp = mu_vec[i*freq:i*freq+train_days,:]
        mu_test = mu_vec[i*freq+train_days:(i+1)*freq+train_days,:]
        gm = GaussianMixture(n_components = n_state, random_state=0).fit(feature_temp)
        all_states = gm.predict(feature_temp)
        #predict the state:
        predicted_state = int(np.round(auto_arima(all_states, start_p=0, start_q=0).predict(1)))
        #get the mu and Sigma, and then optimal weights:
        inds = np.where(all_states==predicted_state) #all the indices for this state

        #if this particular state occurred only once or none, pick the most dominant state (a rare case):
        if len(inds) <= 1:
            predicted_state = np.bincount(all_states).argmax()
            inds = np.where(all_states==predicted_state)

        past_ret = np.take(mu_temp, inds, axis=0)[0] #the returns for this state
        mu = np.mean(past_ret, axis=0)
        Sigma = np.cov(past_ret, rowvar=False)
        
        #get the optimal weights and try 4 solvers 
        try:
            w = opt_w(mu, Sigma, mu.shape[0], 'ECOS')
        except:
            try:
                w = opt_w(mu, Sigma, mu.shape[0], 'ECOS_BB')
            except:
                try:
                    w = opt_w(mu, Sigma, mu.shape[0], 'OSQP')
                except:
                    try:
                        w = opt_w(mu, Sigma, mu.shape[0], 'SCS')
                    except:
                        opt_failed_dates.append(dates[i*freq+train_days])
            
            
        #backtest:
        nav_temp = np.cumprod(np.dot(np.exp(mu_test),w)) #net asset value in the test set
        nav_append = nav_temp*nav[-1]
        nav = np.hstack((nav, nav_append))
        
    return nav, opt_failed_dates

In [19]:
#define the candidate configurations
cand_states = [2,3,4]
cand_mu = [vw_mu, vw_rsi]
mu_str = ['vw_mu', 'vw_rsi']
cand_train_len = [0.5, 1, 2, 3, 4] #training set length or lookback period
cand_freq = [21, 63, 252] #rebalancing 
tasks = []
for state in cand_states:
    for i in range(len(cand_mu)):
        for train_len in cand_train_len:
            for freq in cand_freq:
                if freq <= train_len * 253:
                    tasks.append([state, i, train_len, freq])

print(len(tasks))

84


In [21]:
count = 0
for i in range(len(tasks)):
    task = tasks[i]
    state = task[0]
    feature = cand_mu[task[1]]
    train_len = task[2]
    freq = task[3]
    print('State:'+str(state))
    print('Feature:'+mu_str[task[1]])
    print('Train_Len'+str(train_len))
    print('Frequency'+str(freq))
    nav, opt_failed_dates = backtest(state, feature, train_len, freq)
    nan_arr = np.empty(len(nav)-1)
    nan_arr_dates = np.empty(len(nav)-len(opt_failed_dates))
    nan_arr[:] = np.nan
    nan_arr_dates[:] = np.nan
    yearly_ret = np.hstack((np.power(nav[-1], 253/len(nav))-1, nan_arr))
    sp500_nav = sp500_close[-len(nav):]/sp500_close[-len(nav)]
    sp500_ret = np.hstack((np.power(sp500_nav[-1], 253/len(sp500_nav))-1, nan_arr))
    opt_failed_dates_output = np.hstack((opt_failed_dates, nan_arr_dates))
    df_output = pd.DataFrame({'Dates': dates[-len(nav):], 'NAV':nav, 'SP500 NAV':sp500_nav, 'Yearly Ret':yearly_ret, 
                              'SP Yearly Ret':sp500_ret, 'Opt Failed Dates':opt_failed_dates_output})
    df_output.to_csv('sp500_backtest\SP500_{}_{}_{}_{}.csv'.format(str(state), mu_str[task[1]], str(train_len), str(freq)), index=False)
    count += 1
    print(str(count/len(tasks))+'completed')
    

# yearly_vol = np.hstack((np.std((nav[1:]/nav[:-1]-nav[:-1]))*np.sqrt(253), nan_arr))
# yearly_sr = np.hstack((yearly_ret[0]/yearly_vol[0], nan_arr))
# sp500_vol = np.hstack((np.std((sp500_nav[1:]/sp500_nav[:-1]-nav[:-1]))*np.sqrt(253), nan_arr))
# sp500_sr = np.hstack((sp500_ret[0]/sp500_vol[0], nan_arr))


State:2
Feature:vw_mu
Train_Len0.5
Frequency21


C:\Users\xyyh\anaconda3\envs\bda22\lib\site-packages\cvxpy\problems\problem.py:1333: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


0.011904761904761904completed
State:2
Feature:vw_mu
Train_Len0.5
Frequency63
0.023809523809523808completed
State:2
Feature:vw_mu
Train_Len1
Frequency21


C:\Users\xyyh\anaconda3\envs\bda22\lib\site-packages\cvxpy\problems\problem.py:1333: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


0.03571428571428571completed
State:2
Feature:vw_mu
Train_Len1
Frequency63


...

0.8452380952380952completed
State:4
Feature:vw_rsi
Train_Len0.5
Frequency63


C:\Users\xyyh\anaconda3\envs\bda22\lib\site-packages\cvxpy\problems\problem.py:1333: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


0.8571428571428571completed
State:4
Feature:vw_rsi
Train_Len1
Frequency21
0.8690476190476191completed
State:4
Feature:vw_rsi
Train_Len1
Frequency63
0.8809523809523809completed
State:4
Feature:vw_rsi
Train_Len1
Frequency252
0.8928571428571429completed
State:4
Feature:vw_rsi
Train_Len2
Frequency21
0.9047619047619048completed
State:4
Feature:vw_rsi
Train_Len2
Frequency63
0.9166666666666666completed
State:4
Feature:vw_rsi
Train_Len2
Frequency252
0.9285714285714286completed
State:4
Feature:vw_rsi
Train_Len3
Frequency21


C:\Users\xyyh\anaconda3\envs\bda22\lib\site-packages\cvxpy\problems\problem.py:1333: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


0.9404761904761905completed
State:4
Feature:vw_rsi
Train_Len3
Frequency63
0.9523809523809523completed
State:4
Feature:vw_rsi
Train_Len3
Frequency252
0.9642857142857143completed
State:4
Feature:vw_rsi
Train_Len4
Frequency21
0.9761904761904762completed
State:4
Feature:vw_rsi
Train_Len4
Frequency63
0.9880952380952381completed
State:4
Feature:vw_rsi
Train_Len4
Frequency252
1.0completed
